# Measurements in systems with multiple qubits

In the previous tutorial, we discussed the concept of measurements done on single-qubit systems. 
Building upon those ideas, this tutorial will introduce you to measurements done on multi-qubit systems, and how to implement such measurements in Q#. 
This will include measuring a single qubit in a multi-qubit system, as well as measuring multiple qubits simultaneously. 

We recommend to go through the [tutorial that introduces single qubit system measurements](../SingleQubitSystemMeasurements/SingleQubitSystemMeasurements.ipynb) before starting this one.
$\renewcommand{\ket}[1]{\left\lvert#1\right\rangle}$
$\renewcommand{\bra}[1]{\left\langle#1\right\rvert}$

## Types of measurements on multi-qubit systems
There are several types of measurements you can perform on an $n$-qubit system ($n>1$):
- Measuring all the qubits simultaneously in an orthogonal basis ($2^n$ possible outcomes). As we shall see below, this is a direct generalization of orthogonal basis measurements done in single-qubit systems introduced in the previous tutorial.
- Partial measurement: measuring $m$ qubits out of $n$, for $m<n$ ($2^m$ possible outcomes). Partial measurements involve a partial collapse of the system's wave function, since only some of the qubits are measured.
- Joint measurement: measuring a joint property of all $n$ qubits ($2$ possible outcomes).

We will discuss these concepts in the same order as in the list above.

## Full measurements: measurements in multi-qubit bases
Consider a system consisting of $n\geq1$ qubits. The wave function of such a system belongs to a vector space of dimension $2^n$. Thus, the vector space is spanned by an orthogonal basis, such as the computational basis which consists of the vectors $|0\dotsc0\rangle, \dotsc, |1\dotsc 1\rangle$. For generality, we consider an arbitrary orthonormal basis, which we denote by $\{ |b_0\rangle, |b_1\rangle, \dotsc, |b_{2^n-1}\rangle \}$. 

Then, the state $|\psi\rangle$ of the multi-qubit system can be expressed as a linear combination of the $2^n$ basis vectors $|b_i\rangle$. That is, there exist complex numbers $c_0,c_1,\dotsc, c_{2^n-1}$ such that
 $$
 |\psi\rangle = \sum_{i=0}^{2^n-1} c_i|b_i\rangle \equiv \begin{pmatrix}c_0 \\ c_1 \\ \vdots \\ c_{2^n-1}\end{pmatrix}.
 $$
 In line with the usual convention, we choose the wave function to be normalized, so that $|c_0|^2 + \dotsc + |c_{2^n-1}|^2 =1$. Then, a quantum measurement in the $\{ |b_0\rangle, |b_1\rangle, \dotsc, |b_{2^n-1}\rangle \}$ basis satisfies the following rules:
 - The measurement outcome $b_i$ occurs with probability $|c_i|^2$. 
 - Whenever the measurement outcome is $b_i$, the wave function collapses to the state $|b_i\rangle$. That is, the post-measurement state of the system is equal to $|b_i\rangle$.

This can be summarized in the following table:
<table style="border:1px solid">
    <col width=150>
    <col width=150>
    <col width=150>
    <tr>
        <th style="text-align:center; border:1px solid">Measurement outcome</th>
        <th style="text-align:center; border:1px solid">Probability of outcome</th>
        <th style="text-align:center; border:1px solid">State after measurement</th>
    </tr>
    <tr>
        <td style="text-align:center; border:1px solid">$b_i$</td>
        <td style="text-align:center; border:1px solid">$|c_i|^2$</td>
        <td style="text-align:center; border:1px solid">$\ket{b_i}$</td>
    </tr>    
</table>
 
 
> Similar to measurements in single qubit systems, the assumption of normalization of the original wave function is required in order to ensure that the sum of all the outcome probabilities is 1.

### <span style="color:blue">Exercise 1</span>: Multi-qubit measurement outcome probabilities I 
Suppose that a two-qubit system is known to be in the following state:
$$\ket \psi =  \frac{1}{3}\ket {00} + \frac{2}{3} \ket {01} + \frac{2}{3}\ket {11}$$

If all the qubits are measured simultaneously in the computational basis, what are the outcome probabilities?

*Can't come up with a solution? See the explained solution in the [Multi-Qubit System Measurement Workbook](./Workbook_MultiQubitSystemMeasurements.ipynb#Exercise-1:-Multi-qubit-measurement-outcome-probabilities-I).*

### <span style="color:blue">Exercise 2</span>: Multi-qubit measurement outcome probabilities II 
Suppose that a two-qubit system is known to be in the following state:
$$\ket \psi =  \frac{2}{3}\ket {00} + \frac{1}{3} \ket {01} + \frac{2}{3}\ket {11}$$

If all the qubits are measured simultaneously in the Pauli X basis, i.e., in the $\{ \ket{++}, \ket{+-}, \ket{-+}, \ket{--}\}$ basis, what are the outcome probabilities?

*Can't come up with a solution? See the explained solution in the [Multi-Qubit System Measurement Workbook](./Workbook_MultiQubitSystemMeasurements.ipynb#Exercise-2:-Multi-qubit-measurement-outcome-probabilities-II).*

In [10]:
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Math;

operation HadamardProbabilities(): Unit {
    //
    use register = Qubit[2];

    let theta1 = 2.0 * ArcSin(Sqrt(5.0) / 3.0);
    Ry(theta1, register[1]);

    let theta2 = 2.0 * ArcSin(2.0 / Sqrt(5.0));
    Controlled Ry(register[1..1], (theta2, register[0]));

    // We can save this step by transforming the second qubit first
    //SWAP(register[0], register[1]);

    DumpMachine();

    ApplyToEach(H, register);

    DumpMachine();

    ResetAll(register);
}

HadamardProbabilities

In [11]:
%simulate HadamardProbabilities

|00⟩	0.6666666666666667 + 0𝑖
|01⟩	0.33333333333333337 + 0𝑖
|10⟩	0 + 0𝑖
|11⟩	0.6666666666666665 + 0𝑖

|00⟩	0.8333333333333335 + 0𝑖
|01⟩	-0.16666666666666663 + 0𝑖
|10⟩	0.16666666666666677 + 0𝑖
|11⟩	0.5 + 0𝑖

()

## Measuring each qubit in a system one after another
As described in the previous sections, in theory it is possible to measure all the qubits in an $n$-qubit system simultaneously in an orthogonal basis. The post-measurement state of the qubits is then exactly one of the $2^n$ possible basis states.

In practice, this is implemented by measuring all the qubits one after another. For example, if one wants to measure a two-qubit system in the computational basis, one can implement this by first measuring the first qubit in the computational basis to obtain $0$ or $1$, and then measuring the second qubit in the computational basis. This can result in one of the four possible outcomes: $00, 01, 10, 11$.

This can be generalized to measurements in other bases, such as the 2-qubit Pauli X basis $\ket{++}, \ket{+-}, \ket{-+}, \ket{--}$, and the bases for larger numbers of qubits.

> Note that measuring all qubits one after another can only be used to measure in orthogonal bases $\{ \ket{b_i}\}$ such that each $\ket{b_i}$ is a 'tensor product state'. That is, each $\ket{b_i}$ must be of the form $\ket{v_0} \otimes \ket{v_1} \dotsc \otimes \ket{v_{n-1}}$, with each $\ket{v_j}$ being a single-qubit basis state. 
For example, for the 2-qubit Pauli X basis $\ket{++}, \ket{+-}, \ket{-+}, \ket{--}$ each basis state is a tensor product of states $\ket{+}$ and $\ket{-}$, which form a single-qubit basis state.
> 
> Measuring in orthogonal bases which contain states which are not tensor product states, such as the Bell basis, are trickier to implement, and require appropriate unitary rotations in addition to measuring all qubits one after another. 
> We will not discuss such measurements in this tutorial.
> You can find examples of such measurements and their implementations in the [Measurements kata](../../Measurements/Measurements.ipynb).
>
> If we restrict ourselves to measurements in tensor product states, the distinction between measuring all the qubits simultaneously versus one after another is not important for an ideal quantum computer: in terms of the outcomes and measurement probabilities, both are identical. Furthermore, as long as all the qubits are measured, the sequence in which they are measured is also inconsequential. These factors can be  important in the case of real quantum computers with imperfect qubits, but we restrict the discussion to ideal systems in this tutorial.


### <span style="color:blue">Demo: Measurement statistics for  qubit-by-qubit full measurement</span>
This demo illustrates the equivalence of the measurement probabilities for simultaneous measurement on all qubits, and measurements on each of the qubits executed one after another. Using the wave function from exercise 1 above as an example, we show that the measurement probabilities obtained using the `M` operation in Q# are the same as those expected theoretically for exercise 1. 

In [12]:
// Run this cell using Ctrl+Enter (⌘+Enter on Mac)
// Run the next cell to see the output

open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Preparation;
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Convert;

operation DemoBasisMeasurement(numRuns : Int) : Unit {
    
    // Define coefficients and obtain measurement probabilities
    // for the state corresponding to Exercise 1
    // |𝜓❭ = 0.33 |00❭ + 0.67 |01❭ + 0.67 |11❭ 
    // Use little endian format to encode basis states as integer indices.
    let coefficients = [0.333, 0.0, 0.667, 0.667]; 
    let expected_probabilities = [0.111, 0.0, 0.445, 0.445];
    
    // Set up counter array for measurements.
    mutable countArray = [0, 0, 0, 0]; 
    
    use qs = Qubit[2];

    for i in 1 .. numRuns {

        let register = LittleEndian(qs);
        
        PrepareArbitraryStateD(coefficients, register);

        if (i == 1) {
            Message("The state |𝜓❭ of the system before measurement is:");
            DumpMachine();
        } 

        // Measure the first qubit, followed by the second qubit, and convert the result to little endian integer
        // Returns 0, 1, 2, or 3 depending on little Endian measurement result (never 1)
        let result = MeasureInteger(register);

        // Update countArray
        set countArray w/= result <- countArray[result] + 1; 
    }
    
    // Obtain simulated probability of measurement for each outcome
    mutable simulated_probabilities = [];

    for i in 0 .. 3 {
        set simulated_probabilities += [IntAsDouble(countArray[i]) / IntAsDouble(numRuns)];
    }
    
    Message($"Theoretical measurement probabilities are {expected_probabilities}");
    Message($"Simulated measurement probabilities are {simulated_probabilities}");
}

DemoBasisMeasurement

In [13]:
%simulate DemoBasisMeasurement numRuns=100

The state |𝜓❭ of the system before measurement is:


|00⟩	0.3328890000246523 + 0𝑖
|01⟩	0.6667776667160455 + 0𝑖
|10⟩	1.962615573354719E-17 + 0𝑖
|11⟩	0.6667776667160455 + 0𝑖

Theoretical measurement probabilities are [0.111,0,0.445,0.445]
Simulated measurement probabilities are [0.11,0,0.43,0.46]


()

The simulated probabilities will be different for each run of `DemoBasisMeasurement`. The simulated and theoretical probabilities are not expected to be identical, since measurements are probabilistic. However, we expect the values to be similar, and the simulated probabilities to approach the theoretical probabilities as the parameter `numRuns` is increased.

## Using full measurements to identify the state of the system

Full measurements can also be used to identify the state of the system, if it is guaranteed to be in one of several possible orthogonal states.

### <span style="color:blue">Exercise 3</span>: Distinguish four basis states

**Input:** Two qubits (stored in an array of length 2) which are guaranteed to be in one of the four basis states ($|00\rangle$, $|01\rangle$, $|10\rangle$, or $|11\rangle$).

**Output:**

* 0 if the qubits were in the $|00\rangle$ state,
* 1 if they were in the $|01\rangle$ state, 
* 2 if they were in the $|10\rangle$ state, 
* 3 if they were in the $|11\rangle$ state.

In this task and the subsequent ones the order of qubit states in task description matches the order of qubits in the array (i.e., $|10\rangle$ state corresponds to `qs[0]` in state $|1\rangle$ and `qs[1]` in state $|0\rangle$).

The state of the qubits at the end of the operation does not matter.

In [25]:
%kata T1_BasisStateMeasurement

open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Arrays;

operation BasisStateMeasurement(qs : Qubit[]) : Int {
    // 
    return ResultArrayAsInt(Reversed(MultiM(qs)));
    //let m1 = M(qs[0]) == Zero ? 0 | 1;
    //let m2 = M(qs[1]) == Zero ? 0 | 1;
    //return m1 * 2 + m2;
}

Success!

*Can't come up with a solution? See the explained solution in the [Multi-Qubit System Measurement Workbook](./Workbook_MultiQubitSystemMeasurements.ipynb##Exercise-3:-Distinguish-four-basis-states).*

## Partial Measurements

For a system with $n>1$ qubits, it is possible to measure $m<n$ qubits one after another. The number of measurement outcomes is then $2^m$ instead of $2^n$. The probabilities of each of the outcomes and the post-measurement states of the qubits can be found using the projection formalism for measurements.

First, we recall the concept of [projection operators](../SingleQubitSystemMeasurements/SingleQubitSystemMeasurements.ipynb#Measurements-as-projection-operations) introduced in the single-qubit systems measurements tutorial. Measurements are modeled by orthogonal projection operators - matrices that satisfy
$$
P^2 = P^\dagger = P.
$$
Consider an $n$-qubit system in a state $|\psi\rangle$, for which the first $m<n$ qubits are measured in an orthogonal basis $\{ |b_0\rangle , |b_1\rangle, \dotsc, |b_{2^m-1}\rangle\}$ corresponding to the $m$ qubits being measured. Then we define $2^m$ projectors corresponding to each of the $|b_i\rangle$ states as
$$
P_i = |b_i\rangle \langle b_i| \otimes \mathbb{1}_{n-m},
$$
where $\mathbb{1}_{n-m}$ is the identity operator over the remaining $(n-m)$ qubits. 
> The symbol $\otimes$ represents the tensor product or the Kronecker product of two matrices. It is different from the usual matrix multiplication (see the [Linear Algebra tutorial](../LinearAlgebra/LinearAlgebra.ipynb#Tensor-Product) for a refresher). In the current context, $|b_i\rangle \langle b_i| \otimes \mathbb{1}_{n-m}$ simply means that the operator $|b_i\rangle \langle b_i|$ acts only on the $m$ qubits being measured, while the effect of $P_i$ on the remaining qubits is $\mathbb{1}_{n-m}$, i.e., the identity operator. 

Analogous to the case for measurements for single qubit systems, the rules for partial measurement probabilities and outcomes can be summarized as follows:
- When a measurement is done, one of these projectors is chosen randomly. The probability of choosing projector $P_i$ is $\big|P_i|\psi\rangle\big|^2$.
- If the projector $P_i$ is chosen, the measurement outcome is $b_i$, and the state of the system after the measurement is given by
$$
\frac{P_i |\psi\rangle}{\big|P_i |\psi\rangle\big|}.
$$

For example, consider a two-qubit system in the state $\ket \psi = \frac{1}{\sqrt{2}}\ket{01} - \frac{1}{\sqrt 2}\ket{10}$. Consider a measurement of the first qubit in the computational basis, i.e., in the $\{\ket 0 , \ket 1 \}$ basis. Then, we have two projectors that represent this measurement:
\begin{align*}
P_0 &= \ket 0\bra 0 \otimes \mathbb{1},\\
P_1 &= \ket 1 \bra 1 \otimes \mathbb{1}.
\end{align*}

The action of $P_0$ on $\ket \psi$ is 
\begin{align*}
P_0 \ket \psi &= \left(\ket 0\bra 0 \otimes \mathbb{1}\right) \frac{1}{\sqrt 2}\big(\ket{01} - \ket{10}\big) = \\
              &= \frac{1}{\sqrt 2} \big( \ket 0\bra 0 0\rangle \otimes \mathbb{1} \ket{1} - \ket 0 \bra 0 1\rangle \otimes \mathbb{1} \ket 0 \big) = \\
              &= \frac{1}{\sqrt 2} \ket{01}.
\end{align*}

Similarly, we obtain 
$$
P_1 \ket\psi = -\frac{1}{\sqrt 2} \ket{10}.
$$

Clearly, we have $\big|P_0 \ket \psi\big| = \big|P_1 \ket \psi\big| = \frac{1}{2}$ in this case. Thus, the probabilities of measuring $0$ and $1$ are both $0.5$, with the post-measurement states of system being $\ket{01}$ and $\ket{10}$, respectively.

> Similar to the case of single-qubit system measurements, the applicability of the formalism above requires the state of the multi-qubit system, $\ket \psi$, to be normalized. This is required to ensure that all the probabilities of individual outcomes add up to 1.

### <span style="color:blue">Exercise 4</span>: Partial measurement probabilities for the Hardy state

Consider a 2-qubit system in the state $\ket \psi = \frac{1}{\sqrt{12}} \big(3|00\rangle + |01\rangle + |10\rangle + |11\rangle\big)$.

If only the first qubit is measured in the computational basis, what are the probabilities of the outcomes, and the post-measurement states of the system?

*Can't come up with a solution? See the explained solution in the [Multi-Qubit System Measurement Workbook](./Workbook_MultiQubitSystemMeasurements.ipynb#Exercise-4:-Partial-measurement-probabilities-for-the-Hardy-state).*

### <span style="color:blue">Demo: Measurement statistics for  partial measurement</span>
Using the `M` operation in Q#, we demonstrate that the simulated outcome probabilities and post-measurement outcomes match the theoretical values obtained using the projection operators as described above. We use the Hardy state from Exercise 4 with a computational basis measurement on the first qubit for this purpose.

In [28]:
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Preparation;
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Math;

operation DemoPartialMeasurement(numRuns : Int) : Unit {
    let divider = "--------------------------------------------------------------------------------------------------";
    // 
    // We can use coefficients without normalization in PrepareArbitraryStateD, 
    // the operation will normalize them automatically.
    let coefficients = [3., 1., 1., 1.]; 
    let expected_probabilities = [0.833, 0.167];
    
    // Set up the counter array for measurements.
    mutable countArray = [0, 0];
    
    use qs = Qubit[2];
    for i in 1 .. numRuns {
        // Prepare the state from Exercise 4:
        // |𝜓❭ = (1/√12)(3|00⟩+|01⟩+|10⟩+|11⟩) 
        PrepareArbitraryStateD(coefficients, LittleEndian(qs));
            
        // Display the state of the qubits.
        if (i == 1) {
            Message("The state |𝜓❭ of the system before measurement is:");
            DumpMachine();
            Message(divider);
        }

        // Measure the first qubit.
        let outcome = M(qs[0]) == Zero ? 0 | 1;
        set countArray w/= outcome <- countArray[outcome] + 1;
        
        if (countArray[outcome] == 1) { 
            // The first time the outcome is 0 / 1, print the system state afterwards.
            Message($"For outcome {outcome}, the post-measurement state of the system is:");
            DumpMachine();
        }
        ResetAll(qs);
    }
    
    // Obtain simulated probability of measurement for each outcome
    mutable simulated_probabilities = [];
    for i in 0 .. 1 {
        set simulated_probabilities += [IntAsDouble(countArray[i]) / IntAsDouble(numRuns)];
    }
    
    Message($"Theoretical measurement probabilities are {expected_probabilities}");
    Message($"Simulated measurement probabilities are {simulated_probabilities}");
}

DemoPartialMeasurement

In [29]:
%simulate DemoPartialMeasurement numRuns=100

The state |𝜓❭ of the system before measurement is:


|00⟩	0.8660254037844392 + 0𝑖
|01⟩	0.288675134594813 + 0𝑖
|10⟩	0.28867513459481303 + 0𝑖
|11⟩	0.288675134594813 + 0𝑖

--------------------------------------------------------------------------------------------------
For outcome 1, the post-measurement state of the system is:


|00⟩	0 + 0𝑖
|01⟩	0 + 0𝑖
|10⟩	0.7071067811865476 + 0𝑖
|11⟩	0.7071067811865475 + 0𝑖

For outcome 0, the post-measurement state of the system is:


|00⟩	0.9486832980505139 + 0𝑖
|01⟩	0.3162277660168379 + 0𝑖
|10⟩	0 + 0𝑖
|11⟩	0 + 0𝑖

Theoretical measurement probabilities are [0.833,0.167]
Simulated measurement probabilities are [0.84,0.16]


()

The simulated and theoretical measurement probabilities are not expected to match exactly, but should be close to each other, since measurement is probabilistic. However, the post-measurement states from the simulation should match the expected states for Exercise 4 precisely, since partial state collapse is not a probabilistic process.

## Using partial measurements to identify the state of the system

In certain situations, it is possible to distinguish between orthogonal states of multi-qubit systems using partial measurements, as illustrated in the next exercise.

### <span style="color:blue">Exercise 5</span>: Distinguish orthogonal states using partial measurements

**Input**: A 3-qubit system which is guaranteed to be in either the $\ket{++-}$ state, or the $\ket{---}$ state.

**Output**: 
- 0 if the qubit is in the $\ket{++-}$ state,
- 1 if the qubit is in the $\ket{---}$ state.

The state of the qubits at the end of the operation should be the same as the initial state.

In [35]:
%kata T2_IsPlusPlusMinus

//open Microsoft.Quantum.Measurement;

operation IsPlusPlusMinus (qs : Qubit[]) : Int {
    // This measurement will not change the state of the system, 
    // since the post-projection state is identical to the original state regardless of the input state.
    // For example, if the first qubit was already in the + eigenstate, then measuring it does not change it.
    return Measure([PauliX], [qs[0]]) == Zero ? 0 | 1;
}

Success!

*Can't come up with a solution? See the explained solution in the [Multi-Qubit System Measurement Workbook](./Workbook_MultiQubitSystemMeasurements.ipynb#Exercise-5:-Distinguish-orthogonal-states-using-partial-measurements).*

## Measurements and entanglement

Qubits entanglement has an effect on the measurement statistics of the system. If two qubits are entangled, then their measurement outcomes will be correlated, while separable states (which are by definition not entangled) have uncorrelated measurement outcomes.

> It is useful to revisit the concepts of entanglement and separable states, which were introduced in the [tutorial on multi-qubit systems](../MultiQubitSystems/MultiQubitSystems.ipynb#Entanglement). Consider a system of $n>1$ number of qubits, which we divide into two parts: A, consisting of $m$ qubits, and B, consisting of the remaining $n-m$ qubits. We say that the state $\ket \psi$ of the entire system is separable if it can be expressed as a tensor product of the states of parts A and B: 
$$
\ket \psi = \ket {\phi_A} \otimes \ket{\phi_B}
$$
where $\ket{\phi_A}$ and $\ket{\phi_B}$ are wave functions that describe parts $A$ and $B$, respectively. If it is not possible to express $\ket \psi$ in such a form, then we say that system A is entangled with system B.

Consider a measurement on the subsystem $A$ of a separable state. Let the measurement be done in a basis $\{ \ket{b_0},\dotsc,\ket{b_{2^m-1}}\}$. According to the projection formalism, a projection operator $P_i = \ket{b_i}\bra{b_i} \otimes \mathbb{1}$ is chosen randomly. The corresponding post-measurement state of the system is then given by
\begin{align*}
\ket{\psi}_{i} &\equiv \frac{P_i \ket{\psi}}{\big|P_i \ket{\psi}\big|} = \\
               &= \frac{\ket{b_i}\bra{b_i}\phi_A\rangle \otimes \ket {\phi_B}}{\big|\ket{b_i}\bra{b_i}\phi_A\rangle \otimes \ket {\phi_B}\big|} = \\
               &= \frac{\bra{b_i}\phi_A\rangle \cdot \ket{b_i} \otimes \ket {\phi_B}}{\big|\ket{b_i}\big| \cdot \bra{b_i}\phi_A\rangle \cdot \big| \ket {\phi_B}\big|} = \\
               &= \ket{b_i} \otimes \ket{\phi_B}.
\end{align*}

Thus, the state of subsystem $B$ after the measurement is $\ket{\phi_B}$ independently of the outcome $i$ of the measurement on the first qubit. The results of a subsequent measurement on subsystem $B$, including outcome probabilities, will be independent of the result of the first measurement. In other words, the outcomes of the two measurements will be uncorrelated.

On the other hand, if the system is entangled, then the measurement outcomes will be correlated, in a manner dictated by the bases chosen for the measurements on the two subsystems. The following exercise illustrates this phenomenon.

### <span style="color:blue">Exercise 6</span>: Sequential measurements on an entangled state and a separable state
Consider two two-qubit states:
- The Bell state $|\Phi^{+}\rangle = \frac{1}{\sqrt{2}} \big (|00\rangle + |11\rangle\big)$.
- A state $\ket \Theta = \frac{1}{2} \big( \ket{00} + \ket{01} + \ket{10} + \ket{11} \big)$.

For both states, consider a measurement on the first qubit, followed by a measurement on the second qubit, both done in the computational basis. For which state can we expect the measurement outcomes to be correlated? Verify by calculating the sequential measurement probabilities explicitly for both states. 

*Can't come up with a solution? See the explained solution in the [Multi-Qubit System Measurement Workbook](./Workbook_MultiQubitSystemMeasurements.ipynb#Exercise-6:-Sequential-measurements-on-an-entangled-state-and-a-separable-state).*

> Congratulations! You have learned enough about the measurements to start solving problems from the [Measurements kata](../../Measurements/Measurements.ipynb)!
You can give them a try and return to the rest of this tutorial later, or keep reading.

## State modification using partial measurements

For certain multi-qubit systems prepared in a superposition state, it is possible to use partial measurements to collapse a part of the system to some desired state. 

### <span style="color:blue">Exercise 7</span>: State selection using partial measurements 

**Input**: 
1. A 2-qubit system in the state
$$
|\psi\rangle = \frac{1}{\sqrt{2}} |0\rangle \otimes ( a |0\rangle + b|1\rangle) + \frac{1}{\sqrt{2}} |1\rangle \otimes (b|0\rangle + a |1\rangle),
$$
where the constants $a$ and $b$ satisfying $|a|^2 + |b|^2 = 1$ are unknown.
2. An integer $ind$ which is either $0$ or $1$.

**Goal**: 
- If $ind$ equals 0, the state of the state of the second qubit should be $a|0\rangle + b|1\rangle$
- If $ind$ equals 1, the state of the second qubit should be $b|0\rangle + a |1\rangle$. 

The state of the first qubit at the end does not matter.

In [ ]:
%kata T3_StateSelction

operation StateSelction (qs : Qubit[], ind : Int) : Unit {
    // ...
}

*Can't come up with a solution? See the explained solution in the [Multi-Qubit System Measurement Workbook](./Workbook_MultiQubitSystemMeasurements.ipynb#Exercise-7:-State-selection-using-partial-measurements).*

Any multi-qubit state can be prepared from the $|0...0\rangle$ state using an appropriate combination of quantum gates. 
However, sometimes it is easier and more efficient to prepare a state using partial measurements. 
You could prepare a simpler state involving additional qubits, which, when measured, result in a collapse of the remaining qubits to the desired state with a high probability. This is called **post-selection**, and is particularly useful if it is easier to prepare the pre-measurement state with the extra qubits than to prepare the desired state directly using unitary gates alone. This is demonstrated by the following exercise.

### <span style="color:blue">Exercise 8</span>: State preparation using partial measurements

**Input:** Two qubits (in an array) which are in the state $\ket{00}$.

**Goal:**  Modify the qubits to the state $\frac{1}{\sqrt{3}} \big(|00\rangle + |01\rangle + |10\rangle\big)$ using post-selection.

<br/>
<details>
  <summary><b>Need a hint? Click here</b></summary>
  Consider a 3-qubit state $\frac{1}{2}(|00\rangle + |01\rangle + |11\rangle) \otimes |0\rangle + \frac{1}{2} |11\rangle \otimes |1\rangle$.
  What happens when one measures the third qubit?
</details>

In [ ]:
%kata T4_PostSelection

operation PostSelection (qs : Qubit[]) : Unit {
    // ...
}

*Can't come up with a solution? See the explained solution in the [Multi-Qubit System Measurement Workbook](./Workbook_MultiQubitSystemMeasurements.ipynb#Exercise-8:-State-preparation-using-partial-measurements).*

## Joint measurements

Joint measurements, also known as Pauli measurements, are a generalization of 2-outcome measurements to multiple qubits and other bases. In Q#, joint measurements in Pauli bases are implemented using the [Measure](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.intrinsic.measure) operation. Let's review single-qubit measurements in a different light before discussing joint measurements. 

### Single-qubit Pauli measurement
For single-qubit systems, any measurement corresponding to an orthogonal basis can be associated with a Hermitian matrix with eigenvalues $\pm 1$. The possible measurement outcomes (represented as `Result` in Q#) are the eigenvalues of the Hermitian matrix, and the corresponding projection matrices for the measurement are the projection operators onto the *eigenspaces* corresponding to the eigenvalues. 

For example, consider the computational basis measurement, which can result in outcomes `Zero` or `One` corresponding to states $\ket 0$ and $\ket 1$. This measurement is associated with the Pauli Z operator, which is given by 
$$
Z = \begin{pmatrix} 1 & 0 \\ 0 & -1\end{pmatrix} = \ket{0}\bra{0} - \ket{1}\bra{1}.
$$
The $Z$ operator has two eigenvalues, $1$ and $-1$, with corresponding eigenvectors $\ket{0}$ and $\ket{1}$. A $Z$-measurement is then a measurement in the $\{\ket{0},\ket{1}\}$ basis, with the measurement outcomes being $1$ and $-1$ respectively. In Q#, by convention, an eigenvalue of $1$ corresponds to a `Result` of `Zero`, while an eigenvalue of $-1$ corresponds to a `Result` of `One`.

Similarly, one can implement measurements corresponding to the Pauli X and Y operators. We summarize the various properties below:
<table style="border:1px solid">
    <col width=200>
    <col width=50>
    <col width=100>
    <col width=150>
    <col width=150>
    <tr>
        <th style="text-align:center; border:1px solid">Pauli Operator</th>
        <th style="text-align:center; border:1px solid">Matrix</th>
        <th style="text-align:center; border:1px solid">Eigenvalue</th>
        <th style="text-align:center; border:1px solid">Eigenvector/post-measurement state</th>
        <th style="text-align:center; border:1px solid">Measurement Result in Q#</th>
    </tr>
    <tr>
        <td style="text-align:center; border:1px solid" rowspan="2">$X$</td>
        <td style="text-align:center; border:1px solid" rowspan="2">$\begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix}$</td>
        <td style="text-align:center; border:1px solid">+1</td>
        <td style="text-align:center; border:1px solid">$\ket{+}$</td>
        <td style="text-align:center; border:1px solid">Zero</td>
    </tr><tr>
        <td style="text-align:center; border:1px solid">-1</td>
        <td style="text-align:center; border:1px solid">$\ket{-}$</td>
        <td style="text-align:center; border:1px solid">One</td>
    </tr>
    <tr>
        <td style="text-align:center; border:1px solid" rowspan="2">$Y$</td>
        <td style="text-align:center; border:1px solid" rowspan="2">$\begin{bmatrix} 0 & -i \\ i & 0 \end{bmatrix}$</td>
        <td style="text-align:center; border:1px solid">+1</td>
        <td style="text-align:center; border:1px solid">$\ket{i}$</td>
        <td style="text-align:center; border:1px solid">Zero</td>
    </tr><tr>
        <td style="text-align:center; border:1px solid">-1</td>
        <td style="text-align:center; border:1px solid">$\ket{-i}$</td>
        <td style="text-align:center; border:1px solid">One</td>
    </tr>
    <tr>
        <td style="text-align:center; border:1px solid" rowspan="2">$Z$</td>
        <td style="text-align:center; border:1px solid" rowspan="2">$\begin{bmatrix} 1 & 0 \\ 0 & -1 \end{bmatrix}$</td>
        <td style="text-align:center; border:1px solid">+1</td>
        <td style="text-align:center; border:1px solid">$\ket{0}$</td>
        <td style="text-align:center; border:1px solid">Zero</td>
    </tr><tr>
        <td style="text-align:center; border:1px solid">-1</td>
        <td style="text-align:center; border:1px solid">$\ket{1}$</td>
        <td style="text-align:center; border:1px solid">One</td>
    </tr>
</table>

In general, any measurement on a single qubit which results in two outcomes corresponds to the Hermitian operator $U Z U^\dagger$, for some $2\times 2$ unitary matrix $U$.

Joint measurements are a generalization of this principle for multi-qubit matrices.


### Parity measurements
The simplest joint measurement is a parity measurement. A parity measurement treats computational basis vectors differently depending on whether the number of 1's in the basis vector is even or odd. 

For example, the operator $Z\otimes Z$, or $ZZ$ in short, is the parity measurement operator for a two-qubit system. The eigenvalues $1$ and $-1$ correspond to the subspaces spanned by basis vectors $\{ |00\rangle, |11\rangle \}$ and $\{ |01\rangle, |10\rangle \}$, respectively. That is, when a $ZZ$ measurement results in a `Zero` (i.e. the eigenvalue $+1$), the post-measurement state is a superposition of only those computational basis vectors which have an even number of $1$'s. On the other hand, a result of `One` corresponds to a post-measurement state with only odd parity computational basis vectors.

> Let's see what happens to various two-qubit states after the parity measurement. The $Z \otimes Z$ matrix for two qubits is: 
>
>$$Z \otimes Z = \begin{bmatrix}
    1 & 0 & 0 & 0 \\
    0 & -1 & 0 & 0 \\
    0 & 0 & -1 & 0 \\
    0 & 0 & 0 & 1 \\
\end{bmatrix}$$
>
>When this transformation is applied to a basis state $|00\rangle$, we get
>
> $$\begin{bmatrix}
    1 & 0 & 0 & 0 \\
    0 & -1 & 0 & 0 \\
    0 & 0 & -1 & 0 \\
    0 & 0 & 0 & 1 \\
\end{bmatrix} 
\begin{bmatrix} 1 \\ 0 \\ 0 \\ 0 \\ \end{bmatrix} = 
\begin{bmatrix} 1 \\ 0 \\ 0 \\ 0 \\ \end{bmatrix}$$
>
> Comparing this to the characteristic equation for eigenvectors of $Z \otimes Z$ given by
$ Z \otimes Z |\psi\rangle = \lambda |\psi\rangle$,
it is easy to see that $|00\rangle$ belongs to the $+1$ eigenspace, hence the $Z \otimes Z$ measurement will return `Zero` and leave the state unchanged.
>
> Similarly, it can easily be verified that $|11\rangle$ also belongs to $+1$ eigenspace, while $|01\rangle$ and $|10\rangle$ belong to the $-1$ eigenspace.
> 
> Now, what happens if we apply a $Z \otimes Z$ measurement to a superposition state $\alpha |00\rangle + \beta |11\rangle$? We can see that 
>
> $$\begin{bmatrix}
    1 & 0 & 0 & 0 \\
    0 & -1 & 0 & 0 \\
    0 & 0 & -1 & 0 \\
    0 & 0 & 0 & 1 \\
\end{bmatrix} 
\begin{bmatrix} \alpha \\ 0 \\ 0 \\ \beta \\ \end{bmatrix} = 
\begin{bmatrix} \alpha \\ 0 \\ 0 \\ \beta \\ \end{bmatrix}$$
>
>So this state also belongs to the $+1$ eigenspace, and measuring it will return `Zero` and leave the state unchanged. Similarly, we can verify that an $\alpha |01\rangle + \beta |10\rangle$ state belongs to the $-1$ eigenspace, and measuring it will return `One` without changing the state.

Similarly, a parity measurement on a higher number of qubits can be implemented using a $Z \otimes \dotsc \otimes Z$ measurement.

### <span style="color:blue">Exercise 9</span>: Two-qubit parity measurement

**Inputs**: Two qubits stored in an array which are guaranteed to be either in a superposition of the states $|00\rangle$ and $|11\rangle$ or in a superposition of states $|01\rangle$ and $|10\rangle$.

**Output**: 0 if qubits were in the first superposition, 1 if they were in the second superposition.  
*The state of the qubits at the end of the operation should be the same as the starting state.*

In [ ]:
%kata T5_ParityMeasurement

operation ParityMeasurement (qs : Qubit[]) : Int {
    // ...
}

*Can't come up with a solution? See the explained solution in the [Multi-Qubit System Measurement Workbook](./Workbook_MultiQubitSystemMeasurements.ipynb#Exercise-9:-Two-qubit-parity-measurement).*

### Multi-qubit Pauli measurements
Joint measurement is a generalization of the measurement in the computational basis. 
Pauli measurements can also be generalized to a larger number of qubits. A multi-qubit Pauli measurement corresponds to an operator $M_1 \otimes \dotsc \otimes M_n$, with each $M_i$ being from the set of gates $\{X,Y,Z,I\}$. If at least one of the operators is not the identity matrix, then the measurement can result in two outcomes: a `Result` of `Zero` corresponding to eigenvalue $+1$ and a `Result` of `One` corresponding to the eigenvalue $-1$. The corresponding projection operators are the projections onto the corresponding eigenspaces.

For example, a Pauli/joint measurement corresponding to the $X\otimes Z$ operator can be characterized as follows:
<table style="border:1px solid">
    <col width=50>
    <col width=50>
    <col width=150>
    <col width=250>
    <tr>
        <th style="text-align:center; border:1px solid">Eigenvalue</th>
        <th style="text-align:center; border:1px solid">Measurement Result in Q#</th>
        <th style="text-align:center; border:1px solid">Eigenbasis</th>
        <th style="text-align:center; border:1px solid">Measurement Projector</th>
    </tr>
    <tr>
        <td style="text-align:center; border:1px solid">$+1$</td>
        <td style="text-align:center; border:1px solid">Zero</td>
        <td style="text-align:center; border:1px solid">$\{ \ket{0,+}, \ket{1,-} \}$</td>
        <td style="text-align:center; border:1px solid">$P_{+1} = \ket{0,+}\bra{0,+} +  \ket{1,-} \bra{1,-}$</td>
     </tr>
    <tr>
        <td style="text-align:center; border:1px solid">$-1$</td>
        <td style="text-align:center; border:1px solid">One</td>
        <td style="text-align:center; border:1px solid">$\{ \ket{0,-}, \ket{1,+} \}$</td>
        <td style="text-align:center; border:1px solid">$P_{-1} = \ket{0,-}\bra{0,-} +  \ket{1,+} \bra{1,+}$</td>
     </tr>
 </table>   
 
 The rules for measurements are then the same as those outlined in the [partial measurements section](#Partial-Measurements), with the projection operators in the table.

### <span style="color:blue">Exercise 10</span>: Parity measurement in different basis

Consider a system which is in a state $\alpha |00\rangle + \beta |01\rangle + \beta |10\rangle + \alpha |11\rangle$.

What are the possible outcomes and their associated probabilities, if a measurement in an $XX$ Pauli measurement is done?

*Can't come up with a solution? See the explained solution in the [Multi-Qubit System Measurement Workbook](./Workbook_MultiQubitSystemMeasurements.ipynb#Exercise-10:-Parity-measurement-in-different-basis).*

## Conclusion

Congratulations! You have learned enough to solve problems from the [Measurements kata](../../Measurements/Measurements.ipynb) and the [Joint Measurements kata](../../JointMeasurements/JointMeasurements.ipynb).